In [ ]:
import requests
import pandas as pd
pd.options.display.max_columns = 100
import json

# Get all topicsIds

Les données sont dans un elasticsearch (a priori). On ne peut requêter que les 10000 premiers résultats.
Pour avoir une base complète, on passe par les topics qui pour chacun d'entre eux ne dépassent pas les 10k projets.

In [ ]:
r = requests.post("https://api.tech.ec.europa.eu/search-api/prod/rest/facet?apiKey=SEDIA_NONH2020_PROD&text=***")

In [ ]:
data = r.json()

In [ ]:
for d in data["facets"]:
    if d["rawName"] == "topicAbbreviation":
        topics = d 

In [ ]:
topicsIds = []
for t in topics["values"]:
    topicsIds.append({ "id": t["rawValue"], "name": t["value"] })

# Get all projects

In [ ]:
projects = []

In [ ]:
i = 0
for t in topicsIds:
    i += 1
    if t["id"] == "DT-TRANSFORMATIONS-21-2020":
        print(i)

In [ ]:
for tid in topicsIds:
    print(tid["id"])
    url = "https://api.tech.ec.europa.eu/search-api/prod/rest/search?apiKey=SEDIA_NONH2020_PROD&text=" + tid["id"] + "&pageSize=1&pageNumber=1"
    r = requests.post(url)
    data = r.json()
    print(data["totalResults"], " résultats")
    pages = int(data["totalResults"] / 100) + 1
    for i in range(pages):
        print("page ", i+1)
        url2 = "https://api.tech.ec.europa.eu/search-api/prod/rest/search?apiKey=SEDIA_NONH2020_PROD&text=" + tid["id"] + "&pageSize=100&pageNumber=" + str(i+1)
        r2 = requests.post(url2)
        data2 = r2.json()
        for result in data2["results"]:
            mydict = {}
            for column in result:
                if column != 'metadata':
                    mydict[column] = result[column]
                else:
                    for metadata in result['metadata']:
                        if isinstance(result['metadata'][metadata], list):
                            mydict[metadata] = ";".join(result['metadata'][metadata])
                        else:
                            mydict[metadata] = result['metadata'][metadata]
            projects.append(mydict)

            

On a des doublons

In [ ]:
len(projects)

In [ ]:
df = pd.DataFrame(projects)

In [ ]:
df.shape

Donc on dédoublonne :

In [ ]:
df = df.drop_duplicates(subset=["projectId"], keep="first")

In [ ]:
df.shape

On garde les colonnes intéressantes

In [ ]:
df = df[['reference', 'url', 'title', 'summary',
       'content', 'checksum', 'subTypeOfAction',
       'typeOfAction', 'endDate', 'projectWebsite',
       'deliverables', 'objective',
       'freeKeywords', 'subProgramme',
       'topicAbbreviation', 'overallBudget', 'topicDescription',
       'participants', 'acronym', 'programmePeriod',
       'programmes', 'euContributionRate',
       'euContributionAmount', 'ecSignatureDate', 'topicId',
       'programDescription', 'fundedUnder', 'programAbbreviation', 'projectId',
       'programId', 'startDate', 'publications', 'summaries', 'status',
       'destination',
       'destinationGroup', 'programmeDivision', 'crossCuttingPriorities',
       'mission', 'missionGroup']]

In [ ]:
df["url"] = df["url"].apply(lambda x: x.replace("uri -> ", ""))

On créé des fichiers à part pour certaines propriétés qui sont des tableaux

In [ ]:
df

In [ ]:
for element in ["deliverables", "participants", "publications", "summaries"]:
    print(element)
    arr_element = []
    df_element = df[["reference", "programId", "projectId", element]]
    for index, row in df_element.iterrows():
        if row[element] == row[element]:
            try:
                dict_element = json.loads(row[element])
                for de in dict_element:
                    mydict = de
                    mydict["programId"] = row["programId"]
                    mydict["projectId"] = row["projectId"]
                    mydict["reference"] = row["reference"]
                    arr_element.append(mydict)
            except:
                pass
    dfinter = pd.DataFrame(arr_element)
    dfinter.to_csv("data/" + element + ".csv", index=False, sep=";")


On enlève les propriétés ci-dessus

In [ ]:
df = df[[
    'reference', 'url', 'title', 'summary', 'content', 'checksum',
       'subTypeOfAction', 'typeOfAction', 'endDate', 'projectWebsite',
       'objective', 'freeKeywords', 'subProgramme',
       'topicAbbreviation', 'overallBudget', 'topicDescription',
       'acronym', 'programmePeriod', 'programmes',
       'euContributionRate', 'euContributionAmount', 'ecSignatureDate',
       'topicId', 'programDescription', 'fundedUnder', 'programAbbreviation',
       'projectId', 'programId', 'startDate',
       'status', 'destination', 'destinationGroup',
       'programmeDivision', 'crossCuttingPriorities', 'mission',
       'missionGroup'
]]

On sauvegarde

In [ ]:
df.to_csv("data/projects.csv", index=False, sep=";")

In [ ]:
arr = []
for index, row in df.iterrows():
    mydict = {}
    row = row.to_dict()
    for column in row:
        if column != "postalAddress":
            mydict[column] = row[column]
        else:
            if isinstance(row[column], object):
                for item in eval(row[column]):
                    if item != "countryCode":
                        mydict[column + "." + item] = eval(row[column])[item]
                    else:
                        for item2 in eval(row[column])[item]:
                            mydict[item + "." + item2] = eval(row[column])[item][item2]
    arr.append(mydict)

In [ ]:
df = pd.DataFrame(arr)

In [ ]:
df.to_csv("data/participants.csv", index=False, sep=";")